# Feat 2 with FSL 
## Setup and QC 

In [13]:
import glob
import pandas as pd
import os
from shutil import rmtree


In [14]:

DER_DIR = "/projects/niblab/bids_projects/Experiments/Bevel/derivatives"
SUB_DIR = "/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-*"
SUB_DIRS = sorted(glob.glob(SUB_DIR))

check_subs =['sub-005', 'sub-006', 'sub-011', 'sub-013', 'sub-018', 'sub-030',
       'sub-033', 'sub-035', 'sub-036', 'sub-037', 'sub-039', 'sub-047',
       'sub-052', 'sub-053', 'sub-055', 'sub-067', 'sub-069', 'sub-074',
       'sub-077', 'sub-081', 'sub-086', 'sub-090']

### Write model 2 files

In [15]:
def writeDesignFile(DER_DIR, subID, feat1_folders,tempfsf):
    outpath = os.path.join(DER_DIR,subID, "func/Analysis/feat2/{}_exp2.0".format(subID) )
    print(">>> SETTING DESIGN OUTPATH: ", outpath)
    tempfsf = tempfsf.replace("OUTPUT", outpath)
    #print(FEATS)
    i = 1
    for run_path in feat1_folders:
        #run = run_path.split("/")[-1].split(".")[0].split("_")[1]
        run = "run%s"%i
        #print("> %s : %s"%(run, run_path))
        tempfsf = tempfsf.replace(run, run_path)
        i += 1
    #print(tempfsf)
    OUTFILE_PATH = os.path.join(DER_DIR,subID, "func/Analysis/feat2/%s_design_exp2.0.fsf"%subID)
    print("OUTFILE ------------------------>>>> ", OUTFILE_PATH)
    with open(OUTFILE_PATH, "w") as outfile:
        outfile.write(tempfsf)
    outfile.close()


In [17]:
for x in SUB_DIRS:
    good_runs = 0
    subID =  x.split("/")[-1]
    feat1_folders= glob.glob(os.path.join(DER_DIR, "{}/func/Analysis/feat1/*expanded2.0.feat".format(x)))
    for folder in feat1_folders:
        zstats=glob.glob(os.path.join(folder, "stats/cope*.nii.gz"))
        #print(zstats)
        if not zstats:
            print("BAD FOLDER:\n", folder)
            #rmtree(folder)
        else:
            good_runs += 1
    
        
    # start analysis
    
    ### check for existence of feat2 directory
    FEAT2_DIR = os.path.join(subID, "func/Analysis/feat2")
    if os.path.exists(FEAT2_DIR):
        pass
    else:
        os.makedirs(FEAT2_DIR)
    #print("> STARTING PROGRAM......")
    #print("----------------------->>>>SUBJECT: ",sub)

    if good_runs == 4:
        with open(os.path.join(DER_DIR, "design_files/design2.fsf"), 'r') as infile:
            tempfsf=infile.read()
            writeDesignFile(DER_DIR, subID, feat1_folders,tempfsf)
            infile.close()
        
    elif good_runs == 3:
        with open(os.path.join(DER_DIR, "design_files/design2_3.fsf"), 'r') as infile:
            tempfsf=infile.read()
            writeDesignFile(DER_DIR, subID, feat1_folders,tempfsf)
            infile.close()
    elif good_runs == 2:
        with open(os.path.join(DER_DIR, "design_files/design2_2.fsf"), 'r') as infile:
            tempfsf=infile.read()
            writeDesignFile(DER_DIR, subID, feat1_folders,tempfsf)
            infile.close()
    else:
        pass
    
    

BAD FOLDER:
 /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-001/func/Analysis/feat1/task-prob_run-4_expanded2.0.feat
>>> SETTING DESIGN OUTPATH:  /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-001/func/Analysis/feat2/sub-001_exp2.0
OUTFILE ------------------------>>>>  /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-001/func/Analysis/feat2/sub-001_design_exp2.0.fsf
BAD FOLDER:
 /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-002/func/Analysis/feat1/task-prob_run-4_expanded2.0.feat
>>> SETTING DESIGN OUTPATH:  /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-002/func/Analysis/feat2/sub-002_exp2.0
OUTFILE ------------------------>>>>  /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-002/func/Analysis/feat2/sub-002_design_exp2.0.fsf
BAD FOLDER:
 /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-003/func/Analysis/feat1/task-prob_run-4_expanded2.0.feat
>>> SETTING DESIGN OUTP

BAD FOLDER:
 /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-038/func/Analysis/feat1/task-prob_run-4_expanded2.0.feat
>>> SETTING DESIGN OUTPATH:  /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-038/func/Analysis/feat2/sub-038_exp2.0
OUTFILE ------------------------>>>>  /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-038/func/Analysis/feat2/sub-038_design_exp2.0.fsf
>>> SETTING DESIGN OUTPATH:  /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-039/func/Analysis/feat2/sub-039_exp2.0
OUTFILE ------------------------>>>>  /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-039/func/Analysis/feat2/sub-039_design_exp2.0.fsf
BAD FOLDER:
 /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-040/func/Analysis/feat1/task-prob_run-4_expanded2.0.feat
>>> SETTING DESIGN OUTPATH:  /projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-040/func/Analysis/feat2/sub-040_exp2.0
OUTFILE -----------------

### Run slurm job

In [18]:
bash_file = os.path.join('/projects/niblab/bids_projects/Experiments/Bevel/derivatives/code', 'feat2_bevel.job')

sub_ct = len(SUB_DIRS)
#print(sub_ct)
x=1
shell_cmd = "sbatch --array={}-{}%{} {}".format(1,90, sub_ct, bash_file)
print(shell_cmd)
os.system(shell_cmd)


#subject_set = [x.split("/")[-1].split("-")[1].lstrip("0") for x in subject_set]


# by subject 
#for sub_num in sorted(subject_set):
    #shell_cmd = "sbatch --array={}%1 {}".format(sub_num, bash_file)
    #os.system(shell_cmd)
    #print(shell_cmd)
    

sbatch --array=1-90%85 /projects/niblab/bids_projects/Experiments/Bevel/derivatives/code/feat2_bevel.job


0

In [23]:
!squeue -u nbytes

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


## Quality Check

In [24]:
qc_dict={}

In [25]:
mean_func_ct=0
for sub_dir in SUB_DIRS:
    sub_id=sub_dir.split("/")[-1]
    if sub_id not in qc_dict:
        qc_dict[sub_id] = {}
        
        
    feat2_folder=os.path.join(sub_dir, 'func/Analysis/feat2/{}_exp2.0.gfeat'.format(sub_id))
    print(feat2_folder)
    
    
    mean_func = os.path.join(feat2_folder,'mean_func.nii.gz')
    cope_folders= glob.glob(os.path.join(feat2_folder, "*.feat"))
    cope_folder_ct = len(cope_folders)
    zstat_files = glob.glob(os.path.join(feat2_folder, "*.feat/stats/zstat1.nii.gz"))
    zstat_file_ct = len(zstat_files)
    
    qc_dict[sub_id]["zstat_file_ct"] = zstat_file_ct
    qc_dict[sub_id]["cope_folder_ct"] = cope_folder_ct
    
    """
    # parallize code here
    ct=0
    for zstat in zstat_files:
        ct+=1
        shell_cmd = "cluster -i {} -t 0 > temp_file.txt".format(zstat)
        try:
            #print(shell_cmd)
            cluster = check_output(shell_cmd, shell=True)
            temp=pd.read_csv("temp_file.txt", sep='\t')
            max_vox=temp["MAX"].max()

        except:
            max_vox=None
            print("fail")
        
        
        qc_dict[sub_id]["zstat%i"%ct] = max_vox
        #print(ct, zstat)
    
    if os.path.exists(mean_func):
        mean_func_ct+=1
        qc_dict[sub_id]["mean_func"] = 1

    else:
        pass
    """

/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-001/func/Analysis/feat2/sub-001_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-002/func/Analysis/feat2/sub-002_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-003/func/Analysis/feat2/sub-003_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-004/func/Analysis/feat2/sub-004_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-005/func/Analysis/feat2/sub-005_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-006/func/Analysis/feat2/sub-006_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-007/func/Analysis/feat2/sub-007_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-009/func/Analysis/feat2/sub-009_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-010/func/Analysis/feat2/sub-010_exp2.0.gfeat
/projects/

/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-081/func/Analysis/feat2/sub-081_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-082/func/Analysis/feat2/sub-082_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-083/func/Analysis/feat2/sub-083_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-084/func/Analysis/feat2/sub-084_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-085/func/Analysis/feat2/sub-085_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-086/func/Analysis/feat2/sub-086_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-087/func/Analysis/feat2/sub-087_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-088/func/Analysis/feat2/sub-088_exp2.0.gfeat
/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-089/func/Analysis/feat2/sub-089_exp2.0.gfeat
/projects/

In [26]:
df = pd.DataFrame(qc_dict).T
df.shape

(85, 2)

### Data count check

In [27]:
pd.set_option('display.max_rows', None)
df

,zstat_file_ct,cope_folder_ct
sub-001,23,23
sub-002,23,23
sub-003,23,23
sub-004,23,23
sub-005,23,23
sub-006,23,23
sub-007,23,23
sub-009,23,23
sub-010,23,23
sub-011,23,23


### Subjects with missing data

In [28]:
df[(df==0).any(axis="columns")]

,zstat_file_ct,cope_folder_ct
sub-013,0,0
sub-053,0,0
